<a href="https://colab.research.google.com/github/unmatched78/A2A/blob/main/Copy_of_video_information_extracting%5Bomni-7b-qwen%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Video Information Extracting with Qwen2.5-Omni

This notebook demonstrates how to use Qwen2.5-Omni to obtain information from the video stream.

In [ ]:
!pip install qwen-omni-utils[decord]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 98.5 MB/s eta 0:00:00


In [ ]:
!pip install -U flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 52.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.7.4.post1-cp311-cp311-linux_x86_64.whl size=187831595 sha256=58853b28a5a926cae14402bfd8d4d93a45ebf8f9e79533f37ab09d0d77a99c05
  Stored in directory: /root/.cache/pip/wheels/3d/88/d8/284b89f56af7d5bf366b10d6b8e251ac8a7c7bf3f04203fb4f
Successfully built flash-attn


In [ ]:
!pip uninstall transformers
!pip install git+https://github.com/huggingface/transformers@3a1ead0aabed473eafe527915eea8c197d424356
!pip install accelerate
!pip install qwen-omni-utils
!pip install openai

Found existing installation: transformers 4.50.0
Uninstalling transformers-4.50.0:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.11/dist-packages/transformers-4.50.0.dist-info/*
    /usr/local/lib/python3.11/dist-packages/transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled transformers-4.50.0
  Cloning https://github.com/huggingface/transformers (to revision 3a1ead0aabed473eafe527915eea8c197d424356) to /tmp/pip-req-build-nioo_8fd
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-nioo_8fd
  Running command git rev-parse -q --verify 'sha^3a1ead0aabed473eafe527915eea8c197d424356'
  Running command git fetch -q https://github.com/huggingface/transformers 3a1ead0aabed473eafe527915eea8c197d424356
  Running command git checkout -q 3a1ead0aabed473eafe527915eea8c197d424356
  Resolved https://github.com/huggingface/transformers to commit 3a1ead0aabed473eafe527915eea8c197d424356
  Installin

In [ ]:
from qwen_omni_utils import process_mm_info

# @title inference function
def inference(video_path, prompt, sys_prompt):
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": [
                {"type": "text", "text": prompt},
                {"type": "video", "video": video_path},
            ]
        },
    ]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    # image_inputs, video_inputs = process_vision_info([messages])
    audios, images, videos = process_mm_info(messages, use_audio_in_video=False)
    inputs = processor(text=text, audios=audios, images=images, videos=videos, return_tensors="pt", padding=True)
    inputs = inputs.to(model.device).to(model.dtype)

    output = model.generate(**inputs, use_audio_in_video=False, return_audio=False)

    text = processor.batch_decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    return text

Load model and processors.

In [ ]:
import torch
from transformers import Qwen2_5OmniModel, Qwen2_5OmniProcessor

model_path = "Qwen/Qwen2.5-Omni-7B"
model = Qwen2_5OmniModel.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="flash_attention_2",
)
processor = Qwen2_5OmniProcessor.from_pretrained(model_path)

You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
Qwen2_5OmniToken2WavModel must inference with fp32, but flash_attention_2 only supports fp16 and bf16, attention implementation of Qwen2_5OmniToken2WavModel will fallback to sdpa.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
from IPython.display import Video

#### Question 1

In [ ]:
video_path = "https://images.all-free-download.com/footage_preview/mp4/tiny_wild_bird_searching_for_food_in_nature_6892037.mp4"
prompt = "what  can you see in the video?"

display(Video(video_path, width=640, height=360))

## Use a local HuggingFace model to inference.
response = inference(video_path, prompt=prompt, sys_prompt="You are a helpful assistant.")
print(response[0])

qwen-vl-utils using decord to read video.


system
You are a helpful assistant.
user
what  can you see in the video?
assistant
In the video, a small bird with a mix of brown, orange, and black feathers is seen standing on the grass near a body of water. The bird appears to be foraging or looking for food. The background shows a blurred view of the water, which reflects some colors, and the surrounding area is green with some small plants. The bird moves around slightly, possibly searching for something on the ground.


#### Question 2

In [ ]:
video_path = "https://images.all-free-download.com/footage_preview/mp4/fresh_vegetables_sale_on_market_6891854.mp4"
prompt = "what fruits are present in this video"

display(Video(video_path, width=640, height=360))

## Use a local HuggingFace model to inference.
response = inference(video_path, prompt=prompt, sys_prompt="You are a helpful assistant.")
print(response[0])

system
You are a helpful assistant.
user
what fruits are present in this video
assistant
The video shows a variety of fruits and vegetables at a market. The fruits visible in the video include:

1. Tomatoes
2. Avocados
3. Lemons
4. Dates
5. Green beans
6. Cucumbers

These items are displayed in a market setting, with people walking around and interacting in the background.


#### Question 3

In [ ]:
video_path = "https://images.all-free-download.com/footage_preview/mp4/decorated_wind_chimes_swinging_in_store_6892523.mp4"
prompt = "What is happening in this video"

display(Video(video_path, width=640, height=360))

## Use a local HuggingFace model to inference.
response = inference(video_path, prompt=prompt, sys_prompt="You are a helpful assistant.")
print(response[0])

system
You are a helpful assistant.
user
What is happening in this video
assistant
The video shows a collection of handmade wind chimes and decorative items hanging from the ceiling in a shop or display area. The items are made from various materials such as shells, beads, and other natural elements. The wind chimes are suspended by strings and wires, and they sway gently, likely due to a breeze. The background features a green wall with some additional decorations and items on display. The overall scene suggests a craft or souvenir shop specializing in unique, handcrafted items.


In [ ]:
from qwen_omni_utils import process_mm_info

# @title inference function
def audio_inference(audio_path, prompt, sys_prompt):
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": [
                {"type": "text", "text": prompt},
                {"type": "audio", "audio": audio_path},
            ]
        },
    ]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    print("text:", text)
    # image_inputs, video_inputs = process_vision_info([messages])
    audios, images, videos = process_mm_info(messages, use_audio_in_video=True)
    inputs = processor(text=text, audios=audios, images=images, videos=videos, return_tensors="pt", padding=True)
    inputs = inputs.to(model.device).to(model.dtype)

    output = model.generate(**inputs, use_audio_in_video=True, return_audio=False)

    text = processor.batch_decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    return text

In [ ]:

import librosa

from io import BytesIO
from urllib.request import urlopen

from IPython.display import Audio

In [ ]:
audio_path = "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen2-Audio/audio/1272-128104-0000.flac"
prompt = "Transcribe the English audio into text without any punctuation marks."

audio = librosa.load(BytesIO(urlopen(audio_path).read()), sr=16000)[0]
display(Audio(audio, rate=16000))

## Use a local HuggingFace model to inference.
response = audio_inference(audio_path, prompt=prompt, sys_prompt="You are a speech recognition model.")
print(response[0])

text: ['<|im_start|>system\nYou are a speech recognition model.<|im_end|>\n<|im_start|>user\nTranscribe the English audio into text without any punctuation marks.<|audio_bos|><|AUDIO|><|audio_eos|><|im_end|>\n<|im_start|>assistant\n']


/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:172: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


system
You are a speech recognition model.
user
Transcribe the English audio into text without any punctuation marks.
assistant
mr quilter is the apostle of the middle classes and we are glad to welcome his gospel


In [ ]:
audio_path = "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen2-Audio/audio/1272-128104-0000.flac"
prompt = "Listen to the provided English speech and produce a translation in Chinese text."

audio = librosa.load(BytesIO(urlopen(audio_path).read()), sr=16000)[0]
display(Audio(audio, rate=16000))

## Use a local HuggingFace model to inference.
response = audio_inference(audio_path, prompt=prompt, sys_prompt="You are a speech translation model.")
print(response[0])

text: ['<|im_start|>system\nYou are a speech translation model.<|im_end|>\n<|im_start|>user\nListen to the provided English speech and produce a translation in Chinese text.<|audio_bos|><|AUDIO|><|audio_eos|><|im_end|>\n<|im_start|>assistant\n']


/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:172: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


system
You are a speech translation model.
user
Listen to the provided English speech and produce a translation in Chinese text.
assistant
奎尔特先生是中产阶级的使徒,我们很高兴欢迎他的福音。


In [ ]:
audio_path = "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen2.5-Omni/cough.wav"
prompt = "Classify the given human vocal sound in English."

audio = librosa.load(BytesIO(urlopen(audio_path).read()), sr=16000)[0]
display(Audio(audio, rate=16000))

## Use a local HuggingFace model to inference.
response = audio_inference(audio_path, prompt=prompt, sys_prompt="You are a vocal sound classification model.")
print(response[0])

text: ['<|im_start|>system\nYou are a vocal sound classification model.<|im_end|>\n<|im_start|>user\nClassify the given human vocal sound in English.<|audio_bos|><|AUDIO|><|audio_eos|><|im_end|>\n<|im_start|>assistant\n']


/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:172: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


system
You are a vocal sound classification model.
user
Classify the given human vocal sound in English.
assistant
Cough
